In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
api_keypath="/content/drive/MyDrive/kaggle/kaggle.json"

In [3]:
!mkdir -p ~/.kaggle
!cp "{api_keypath}" ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle

In [4]:
import os
print(os.getcwd())


/content


In [5]:
!ls -a ~

.   .bashrc  .config   .julia	 .kaggle  .launchpadlib  .npm	   .tmux.conf
..  .cache   .ipython  .jupyter  .keras   .local	 .profile  .wget-hsts


In [6]:
import pandas as pd
import numpy as np

In [7]:
!kaggle competitions download -c playground-series-s5e8

  0% 0.00/14.7M [00:00<?, ?B/s]
100% 14.7M/14.7M [00:00<00:00, 707MB/s]


In [8]:
!unzip playground-series-s5e8.zip

Archive:  playground-series-s5e8.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [9]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [10]:
#データのコピー
train_copydf = train_df.copy()
test_copydf = test_df.copy()

In [ ]:

def encode(df):
  #年齢の正規化
  age_max = df['age'].max()
  df['age_normalized'] = train_df['age'] / age_max
  df=df.drop('age',axis=1)
  #婚姻状態の符号化
  binary_map={"no":0,"yes":1}
  marital_map={"married":1,"single":0}
  df['marital_encoded']=df['marital'].map(marital_map)
  df=df.drop('marital',axis=1)

  # poutcomeカラムをOne-Hotエンコーディング
  # prefix='poutcome'で新しいカラム名の接頭辞を指定
  # drop_first=Trueで、元のカテゴリ数-1のダミー変数を作成（多重共線性を防ぐため）
  poutcome_encoded = pd.get_dummies(df['poutcome'], prefix='poutcome', drop_first=True)
  #job列のone-hot encoding
  job_dummies = pd.get_dummies(df['job'], prefix='job', dummy_na=False)
  df= pd.concat([df,job_dummies],axis=1)
  df=df.drop('job',axis=1)
  #教育状態のone-hot encoding
  education_dummies= pd.get_dummies(df['education'], prefix='education', dummy_na=False)
  df['default_encoded']=df['default'].map(binary_map)
  df=df.drop('education',axis=1)
  df=df.drop('default',axis=1)
  #balance(所得)の対数変換
  min_balance = df['balance'].min()
  if min_balance < 0:
    df['balance_log'] = np.log1p(df['balance'] - min_balance )
  else:
    df['balance_log'] = np.log1p(df['ybalance'])
  df=df.drop('balance',axis=1)
  #housing
  df['housing_encoded']=df['housing'].map(binary_map)
  df=df.drop('housing',axis=1)
  #loan
  df['loan_encoded']=df['loan'].map(binary_map)
  df=df.drop('loan',axis=1)
  #contactのone-hot encoding
  contact_dummies= pd.get_dummies(df['contact'], prefix='contact', dummy_na=False)
  df= pd.concat([df,contact_dummies],axis=1)
  df=df.drop('contact',axis=1)
  #day,monthの周期性を考慮した三角関数エンコーディング
  df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
  df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)

  month_map = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
             'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
  df['month']= df['month'].map(month_map)
  df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
  df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
  df=df.drop(['day','month'],axis=1)
  #duration(通話時間)の対数変換
  df['duration_log']=np.log1p(df['duration'])
  df=df.drop('duration',axis=1)
  #campaign(キャンペーン実施回数)の対数変換
  df['campaign_log']=np.log1p(df['campaign'])
  df=df.drop('campaign',axis=1)
  #pdays
  #999かどうかを判断する二値特徴量を作成
  df['pdays_contacted'] = np.where(df['pdays'] == -1, 0, 1)
  # -1以外の場合に対数変換を適用し、-1の場合は0にする
  df['pdays_log'] = np.where(df['pdays'] == -1, 0, np.log1p(df['pdays']))
  # 元のpdays列は削除
  df=df.drop('pdays',axis=1)
  #previous
  df['previous_log']=np.log1p(df['previous'])
  df=df.drop('previous',axis=1)

  return df




In [ ]:
df_en1=encode(train_copydf)
print(df_en1.head(25))

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


    id poutcome  y  age_normalized  marital_encoded  job_admin.  \
0    0  unknown  0        0.442105              1.0       False   
1    1  unknown  0        0.400000              1.0       False   
2    2  unknown  0        0.378947              1.0       False   
3    3  unknown  0        0.284211              0.0       False   
4    4  unknown  1        0.273684              1.0       False   
5    5  unknown  0        0.252632              0.0        True   
6    6  unknown  0        0.410526              1.0       False   
7    7  unknown  0        0.526316              0.0        True   
8    8  unknown  0        0.484211              1.0       False   
9    9  unknown  0        0.410526              NaN       False   
10  10  unknown  0        0.336842              0.0       False   
11  11  unknown  0        0.389474              1.0       False   
12  12  unknown  0        0.484211              1.0       False   
13  13  unknown  0        0.336842              1.0       Fals

In [ ]:
train_copydf.head(25)

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,age_normalized
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0,0.442105
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0,0.400000
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0,0.378947
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0,0.284211
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1,0.273684
5,5,24,admin.,single,secondary,no,1882,yes,no,cellular,20,apr,1010,3,-1,0,unknown,0,0.252632
6,6,39,blue-collar,married,secondary,no,0,no,no,telephone,21,nov,90,1,-1,0,unknown,0,0.410526
7,7,50,admin.,single,secondary,no,1595,no,no,telephone,31,jul,49,25,-1,0,unknown,0,0.526316
8,8,46,blue-collar,married,primary,no,1463,no,no,cellular,4,aug,50,1,-1,0,unknown,0,0.484211
9,9,39,management,divorced,tertiary,no,25,yes,no,cellular,8,may,119,1,-1,0,unknown,0,0.410526


In [ ]:
test_copydf.head(25)

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,750000,32,blue-collar,married,secondary,no,1397,yes,no,unknown,21,may,224,1,-1,0,unknown
1,750001,44,management,married,tertiary,no,23,yes,no,cellular,3,apr,586,2,-1,0,unknown
2,750002,36,self-employed,married,primary,no,46,yes,yes,cellular,13,may,111,2,-1,0,unknown
3,750003,58,blue-collar,married,secondary,no,-1380,yes,yes,unknown,29,may,125,1,-1,0,unknown
4,750004,28,technician,single,secondary,no,1950,yes,no,cellular,22,jul,181,1,-1,0,unknown
5,750005,43,management,divorced,unknown,no,3025,no,no,cellular,21,jul,89,2,-1,0,unknown
6,750006,26,services,single,primary,no,3511,no,no,cellular,29,jan,816,3,-1,0,unknown
7,750007,60,management,married,tertiary,no,79,no,yes,cellular,9,jul,707,1,-1,0,unknown
8,750008,45,blue-collar,married,primary,no,16,no,no,telephone,6,feb,173,1,-1,0,unknown
9,750009,41,management,married,tertiary,no,46,no,no,cellular,18,aug,657,4,-1,0,unknown


In [ ]:
dftest_en1=encode(test_copydf)
print(dftest_en1.head(25))

        id poutcome  age_normalized  marital_encoded  job_admin.  \
0   750000  unknown        0.442105              1.0       False   
1   750001  unknown        0.400000              1.0       False   
2   750002  unknown        0.378947              1.0       False   
3   750003  unknown        0.284211              1.0       False   
4   750004  unknown        0.273684              0.0       False   
5   750005  unknown        0.252632              NaN       False   
6   750006  unknown        0.410526              0.0       False   
7   750007  unknown        0.526316              1.0       False   
8   750008  unknown        0.484211              1.0       False   
9   750009  unknown        0.410526              1.0       False   
10  750010  unknown        0.336842              0.0       False   
11  750011    other        0.389474              NaN       False   
12  750012  unknown        0.484211              1.0       False   
13  750013  unknown        0.336842             

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
